In [67]:
import pandas as pd

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Base estados brasil
brasil = pd.read_excel('ibge.xlsx', encoding='utf-8',  error_bad_lines=False)

# Base mundo
mundo = pd.read_csv('population_by_country_2020.csv', encoding='utf-8')

# Base de IDH 
idh_mundo = pd.read_csv('Human Development Index (HDI).csv')
idh_mundo['2018'] = pd.to_numeric(idh_mundo['2018'])
idh_mundo = idh_mundo.sort_values(by=['2018'], ascending=False)
idh_mundo = idh_mundo.fillna(0)

# Mudando o título do mundo
mundo = mundo.rename(columns={'Country (or dependency)': 'Country'})


# #Escolha o estado
print("""A lista de estados são: 
0 - Acre
1 - Alagoas
2 - Amapá
3 - Amazonas
4 - Bahia
5 - Ceará
6 - Distrito Federal
7 - Espírito Santo
8 - Goiás
9 - Maranhão
10 - Mato Grosso
11 - Mato Grosso do Sul
12- Minas Gerais
13 - Pará
14 - Paraíba
15 - Paraná
16 - Pernambuco
17 - Piauí
18 - Rio de Janeiro
19 - Rio Grande do Norte
20 - Rio Grande do Sul
21 - Rondônia
22 - Roraima
23 - Santa Catarina
24 - São Paulo
25 - Sergipe
26 - Tocantins
""")
# Pergunta do estado
x=0
while x==0:
    try:
        estado_escolha = int(input("Qual estado você quer comparar? (número) "))
        tolerancia_area = float(input("Porcentagem de tolerância para ÁREA (%): "))
        tolerancia_den = float(input("Porcentagem de tolerância para DENSIDADE (%): "))
        tolerancia_pop = float(input("Porcentagem de tolerância para POPULAÇÃO (%): "))
        tolerancia_idh = float(input("Porcentagem de tolerância para IDH (%): "))
        x=1
    except:
        print('\nValor inválido. Repita: ')
        x=0

# Calculando intervalos de tolerância
area_min = brasil.iloc[estado_escolha, 5]-tolerancia_area*brasil.iloc[estado_escolha, 5]/100
area_max = brasil.iloc[estado_escolha, 5]+tolerancia_area*brasil.iloc[estado_escolha, 5]/100

populacao_min = brasil.iloc[estado_escolha, 6] - tolerancia_pop*brasil.iloc[estado_escolha, 6]/100
populacao_max = brasil.iloc[estado_escolha, 6] + tolerancia_pop*brasil.iloc[estado_escolha, 6]/100

tolerancia_den_min = brasil.iloc[estado_escolha, 7] - tolerancia_den*brasil.iloc[estado_escolha, 7]/100
tolerancia_den_max = brasil.iloc[estado_escolha, 7] + tolerancia_den*brasil.iloc[estado_escolha, 7]/100

idh_min = brasil.iloc[estado_escolha, 9] - tolerancia_idh*brasil.iloc[estado_escolha, 9]/100
idh_max = brasil.iloc[estado_escolha, 9] + tolerancia_idh*brasil.iloc[estado_escolha, 9]/100

# Subtração valores
mundo['difArea'] = abs(mundo['Land Area (Km²)'] - brasil.iloc[estado_escolha, 5])
mundo['difPopulacao'] = abs(mundo['Population (2020)'] - brasil.iloc[estado_escolha, 6])
mundo['difDensidade'] = abs(mundo['Density (P/Km²)'] - brasil.iloc[estado_escolha, 7])
mundo['resultante'] = mundo['difArea'] * mundo['difPopulacao'] * mundo['difDensidade']
idh_mundo['difIDH'] = abs(idh_mundo['2018'] - brasil.iloc[estado_escolha, 9])


# Extrair valores
print('--------------------------------------------------------')
print('\n - ÁREA PRÓXIMA: ')
print('área oficial (km²): ' + str(brasil.iloc[estado_escolha, 5]))
mundo[(mundo['Land Area (Km²)'] >= area_min) & (mundo['Land Area (Km²)'] <= area_max)]
print('País com área mais próxima: ' + str(mundo[mundo['difArea']==mundo['difArea'].min()].iloc[0,0]))
print('Valor percentual da diferença: ' + str(100*mundo['difArea'].min()/brasil.iloc[estado_escolha, 5]))

print('\n - POPULAÇÃO PRÓXIMA: ')
print('população oficial: ' + str(brasil.iloc[estado_escolha, 6]))
mundo[(mundo['Population (2020)'] >= populacao_min) & (mundo['Population (2020)'] <= populacao_max)]
print('País com população mais próxima: ' + str(mundo[mundo['difPopulacao']==mundo['difPopulacao'].min()].iloc[0,0]))
print('Valor percentual da diferença: ' + str(100*mundo['difPopulacao'].min()/brasil.iloc[estado_escolha, 6]))

print('\n - DENSIDADE POPULACIONAL PRÓXIMA: ')
print('densidade populacional oficial (hab./km²): ' + str(brasil.iloc[estado_escolha, 7]))
mundo[(mundo['Density (P/Km²)'] >= tolerancia_den_min) & (mundo['Density (P/Km²)'] <= tolerancia_den_max)]
print('País com densidade mais próxima: ' + str(mundo[mundo['difDensidade']==mundo['difDensidade'].min()].iloc[0,0]))
print('Valor percentual da diferença: ' + str(100*mundo['difDensidade'].min()/brasil.iloc[estado_escolha, 7]))

print('\n - IDH PRÓXIMO: ')
print('IDH oficial: ' + str(brasil.iloc[estado_escolha, 9]))
idh_resumo = idh_mundo[(idh_mundo['2018'] >= idh_min) & (idh_mundo['2018'] <= idh_max)].reset_index()
idh_resumo[['HDI Rank (2018)', 'Country', '2018', 'difIDH']]
print("País com mais parecido com o estado escolhido com relação IDH: " + str(idh_resumo[idh_resumo['difIDH']==idh_resumo['difIDH'].min()].iloc[0,2]))
print('Valor percentual da diferença: ' + str(100*idh_resumo['difIDH'].min()/brasil.iloc[estado_escolha, 9]))

# Valores finais mais próximos
print('\n')
print("País com mais parecido com o estado escolhido com relação ÁREA, POPULAÇÃO, DENSIDADE: ")
mundo[mundo['resultante']==mundo['resultante'].min()]

# Resumo dados Brasil
print('--------------------------------------')
print('Área média dos estados brasileiros (km²): ' + str(brasil['Área Territorial - km² [2018]'].mean()))
print('Número de letras médio do nome dos estados: ' + str(brasil['UF [-]'].map(len).mean()))
print('Porcentagem gentílicos terminados por -ense (%): ' + str(round(100*brasil['Gentílico [-]'].str.contains('ense', regex=False).value_counts()[1]/brasil['Gentílico [-]'].str.contains('ense', regex=False).value_counts().sum(), 2)))
print('População média dos estados: ' + str(brasil['População estimada - pessoas [2019]'].mean()))

A lista de estados são: 
0 - Acre
1 - Alagoas
2 - Amapá
3 - Amazonas
4 - Bahia
5 - Ceará
6 - Distrito Federal
7 - Espírito Santo
8 - Goiás
9 - Maranhão
10 - Mato Grosso
11 - Mato Grosso do Sul
12- Minas Gerais
13 - Pará
14 - Paraíba
15 - Paraná
16 - Pernambuco
17 - Piauí
18 - Rio de Janeiro
19 - Rio Grande do Norte
20 - Rio Grande do Sul
21 - Rondônia
22 - Roraima
23 - Santa Catarina
24 - São Paulo
25 - Sergipe
26 - Tocantins

Qual estado você quer comparar? (número) 1
Porcentagem de tolerância para ÁREA (%): 1
Porcentagem de tolerância para DENSIDADE (%): 1
Porcentagem de tolerância para POPULAÇÃO (%): 1
Porcentagem de tolerância para IDH (%): 1
--------------------------------------------------------

 - ÁREA PRÓXIMA: 
área oficial (km²): 27843.295


,Country,Population (2020),Yearly Change,Net Change,Density (P/Km²),Land Area (Km²),Migrants (net),Fert. Rate,Med. Age,Urban Pop %,World Share,difArea,difPopulacao,difDensidade,resultante
149,Guinea-Bissau,1958132,2.45 %,47079,70,28120,-1399.0,4.5,19,45 %,0.03 %,276.705,1379225,42.33,1.615476e+10
152,Equatorial Guinea,1392950,3.47 %,46999,50,28050,16000.0,4.6,22,73 %,0.02 %,206.705,1944407,62.33,2.505159e+10
165,Solomon Islands,683301,2.55 %,17061,25,27990,-1600.0,4.4,20,23 %,0.01 %,146.705,2654056,87.33,3.400310e+10


País com área mais próxima: Solomon Islands
Valor percentual da diferença: 0.5268952543152732

 - POPULAÇÃO PRÓXIMA: 
população oficial: 3337357


,Country,Population (2020),Yearly Change,Net Change,Density (P/Km²),Land Area (Km²),Migrants (net),Fert. Rate,Med. Age,Urban Pop %,World Share,difArea,difPopulacao,difDensidade,resultante


País com população mais próxima: Bosnia and Herzegovina
Valor percentual da diferença: 1.5746292650142013

 - DENSIDADE POPULACIONAL PRÓXIMA: 
densidade populacional oficial (hab./km²): 112.33


...

País com densidade mais próxima: Portugal
Valor percentual da diferença: 1.1840113949968827

 - IDH PRÓXIMO: 
IDH oficial: 0.631


,HDI Rank (2018),Country,2018,difIDH
0,131,Timor-Leste,0.626,0.005


País com mais parecido com o estado escolhido com relação IDH: Timor-Leste
Valor percentual da diferença: 0.7923930269413636


País com mais parecido com o estado escolhido com relação ÁREA, POPULAÇÃO, DENSIDADE: 


,Country,Population (2020),Yearly Change,Net Change,Density (P/Km²),Land Area (Km²),Migrants (net),Fert. Rate,Med. Age,Urban Pop %,World Share,difArea,difPopulacao,difDensidade,resultante
139,Albania,2878420,-0.11 %,-3120,105,27400,-14000.0,1.6,36,63 %,0.04 %,443.295,458937,7.33,1.491248e+09


--------------------------------------
Área média dos estados brasileiros (km²): 315215.5786296296
Número de letras médio do nome dos estados: 9.407407407407407
Porcentagem gentílicos terminados por -ense (%): 70.37
População média dos estados: 7783226.851851852
